In [1]:
import requests
import pandas as pd
import html
from lxml import etree
import re
import time
import csv
import json

In [131]:
# requests.adapters.DEFAULT_RETRIES = 5
test_cookie = "fvlid=1637065968543gFqWW84A3Sr8; sessionid=4e7a7b9b-5542-442f-8fe7-aef458c1e76c; che_sessionid=44E02423-8651-4DC2-B69B-783B9804CD06%7C%7C2021-11-16+20%3A32%3A49.771%7C%7C0; pcpopclub=5463925b3e994bb6918e42aa151d55180cfcb4c5; clubUserShow=217887941|0|2|%e5%8d%97%e4%ba%ac%e8%bd%a6%e5%8f%8bnigdgi|0|0|0|/g24/M06/CA/50/120X120_0_q87_autohomecar__ChsEeV9HeBOAYafcAACLZ1VUaU0317.jpg|2021-11-18 10:58:30|0; sessionip=112.20.84.78; area=320199; Hm_lvt_d381ec2f88158113b9b76f14c497ed48=1642165793; memberPhoneInfo=%7C13776656469%7C1; SessionSeries=0; searchhistoryval=%E5%A5%94%E9%A9%B0CLA%202014%E6%AC%BE%20CLA%20260%204MATIC; usedcaruid=1+gBCh238eyx08wTSFNFxA==; sessionvisit=02a51126-32a3-49b2-a5ca-16d773f8122e; sessionvisitInfo=4e7a7b9b-5542-442f-8fe7-aef458c1e76c|www.che168.com|104888; che_sessionvid=5F013099-CFAA-4227-A3DA-73996E1218BF; UsedCarBrowseHistory=0%3A39618795%2C0%3A34271494%2C0%3A39511215%2C0%3A42751078%2C0%3A39525696%2C0%3A42825291%2C0%3A42729964%2C0%3A42606522%2C0%3A42710780%2C0%3A42275443; userarea=320100; sessionuid=4e7a7b9b-5542-442f-8fe7-aef458c1e76c; showNum=251; ahpvno=55; Hm_lpvt_d381ec2f88158113b9b76f14c497ed48=1642412238; ahuuid=4CB86F0C-BF57-4A4D-A2A2-543DC6379058; v_no=178; visit_info_ad=44E02423-8651-4DC2-B69B-783B9804CD06||5F013099-CFAA-4227-A3DA-73996E1218BF||-1||-1||178; che_ref=0%7C0%7C0%7C0%7C2022-01-17+17%3A37%3A18.410%7C2021-11-16+20%3A32%3A49.771"

send_headers = {
#     "User-Agent": random.choice(agents),
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "zh-CN,zh;q=0.8",
    "Cookie":test_cookie
}
def get_html(info_id):
    url = f'https://www.che168.com/CarConfig/CarConfig.html?infoid={info_id}'
    r = requests.get(url, verify=True, headers=send_headers)
    r_html = html.unescape(r.text)    
    r.close()
    
    return r_html

In [140]:
def get_json(sid):
    url_config = f'https://cacheapigo.che168.com/CarProduct/GetParam.ashx?specid={sid}&callback=configTitle'
    r = requests.get(url_config, timeout=2, verify=False)
    r_html = html.unescape(r.text)
    r.close()
    r_json = json.loads(r_html[12:-1])
    return r_json

In [158]:
def get_carlist(city):
    url = f'https://www.che168.com/{city}/list/'
    r = requests.get(url, timeout=2, verify=False)
    r_html = html.unescape(r.text)
    r.close()
    return r_html

In [159]:
def get_carID(city, page):
    url = f'https://www.che168.com/{city}/a0_0msdgscncgpi1ltocsp{page}exx0/?pvareaid=102179#currengpostion'
    r = requests.get(url, timeout=2, verify=False)
    r_html = html.unescape(r.text)
    r.close()
    return r_html

In [171]:
ID = [86, 87, 88, 90, 91, 92, 95, 115, 101, 108, 116, 104]
city_list = ["shanghai", "guangzhou", "tianjin", "nanjing", "chengdu", "hangzhou",
            "jinan", "zhengzhou", "shijiazhuang", "chongqing", "xian", "shenzhen", "wuhan", 
            "shenyang", "kunming", "hefei", "changsha", "nanning", "haerbin", "dongguan", 
            "guiyang", "wenzhou", "foshan", "suzhou", "changchun", "ningbo", "taiyuan", "wulumuqi",
            "qingdao", "beijing"]

In [5]:
# 列名
with open('whole_car.csv', 'a+', newline='',encoding='utf-8-sig') as f:
    csv_write = csv.writer(f)
    columns = ["Brand","Series","CarName","Province","City","Miles", "Date", "二手车报价" ,"新车厂商指导价", "级别", "能源类型", "环保标准", "上市时间", "最大功率(kW)",  "最大扭矩(N・m)", "发动机", "变速箱", "长*宽*高(mm)", "车身结构", "NEDC综合油耗(L/100km)"]
    csv_write.writerow(columns)
f.close()

In [6]:
import warnings
import random
warnings.filterwarnings("ignore")

In [ ]:
for c in city_list:
    print("正在爬取" + str(c) +"市")
    car_list_html = get_carlist(c)
    t_car_list_html = etree.HTML(car_list_html)
    total_page = t_car_list_html.xpath('.//input[@id="TotalPage"]/@value')
    print(total_page[0])
    total_page = int(total_page[0])
    if (total_page > 100):
        total_page = 100
    for page in range(1, total_page + 1):
        while 1: 
            car_id_html = get_carID(c, page)
            t_car_id_html = etree.HTML(car_id_html)
            divs = t_car_id_html.xpath('.//div[@class="tp-cards-tofu fn-clear"]')
            if len(divs) == 0:
                time.sleep(5)
            else:
                break
        infoid = divs[0].xpath('//li/@infoid')
        print("爬取" + str(len(infoid)) + "个InfoID")
        
        for index in infoid:
#             time.sleep(1 + (random.randint(0,9) / 10))
            print("正在尝试爬取infoID :" + str(index))
            while 1:
                raw_html = get_html(index)
                t_html = etree.HTML(raw_html)
                if not t_html:
                    continue
                fail_or_not = t_html.xpath('.//div[@class="fail-title"]')
                if (len(fail_or_not) == 0):
                    break
                else :
                    print("太快了！准备等待重新爬取第" + str(index) + "个页面")
                    time.sleep(10 + random.randint(0,9))


            brand_name = t_html.xpath('.//div[@class="breadnav content"]/a[@target="_blank"]')
            basic_info = t_html.xpath('.//div[@class="source-info-con"]/p')
            price = t_html.xpath('.//div[@class="source-info-con"]/p[@class="sp-margin"]/span')

            specid = t_html.xpath('.//input[@id="CarSpecid"]/@value')
            if(len(brand_name) == 0) :
                continue
            print("Find it!")

            try:
                # 处理直辖市
                if (len(brand_name) == 7) :
                    province = brand_name[1].text
                    city = brand_name[2].text.replace("二手车","")
                    brand = brand_name[3].text.replace("二手","")
                    series = brand_name[4].text.replace("二手","")
                    car_name = brand_name[5].text
                else:
                    province = brand_name[1].text
                    city = province
                    brand = brand_name[2].text.replace("二手","")
                    series = brand_name[3].text.replace("二手","")
                    car_name = brand_name[4].text


                miles = basic_info[0].text.split('／')[0].replace("万公里","")
                date = basic_info[0].text.split('／')[1]
                price = price[0].text.replace("￥","")

                car_config = get_json(specid[0])
                result = {}

                for item in car_config["result"]["paramtypeitems"][0]['paramitems']:
                    if item['id'] in ID:
                        a = item['name']
                        b = item['value']
                        result[a] = b

                # 有些车型信息不全直接丢弃
                if len(result) == len(ID):
                    print("输出输出")
                    with open("whole_car.csv", 'a', encoding = "utf-8") as f:
                        csv_write = csv.writer(f, delimiter=',')
                        data_row = [brand, series, car_name, province, city, miles, date, price]
                        for data in result.values():
                            data_row.append(data)
                        csv_write.writerow(data_row)
            except:
                pass






正在爬取shanghai市
157
爬取80个InfoID
正在尝试爬取infoID :42210675
Find it!
输出输出
正在尝试爬取infoID :42629148
Find it!
输出输出
正在尝试爬取infoID :42854300
Find it!
输出输出
正在尝试爬取infoID :42490046
Find it!
输出输出
正在尝试爬取infoID :42556267
Find it!
输出输出
正在尝试爬取infoID :42239544
Find it!
输出输出
正在尝试爬取infoID :41777228
Find it!
输出输出
正在尝试爬取infoID :42766456
Find it!
输出输出
正在尝试爬取infoID :42091740
Find it!
输出输出
正在尝试爬取infoID :42853542
Find it!
输出输出
正在尝试爬取infoID :42877476
Find it!
输出输出
正在尝试爬取infoID :42868462
Find it!
正在尝试爬取infoID :42746365
Find it!
正在尝试爬取infoID :42613542
Find it!
输出输出
正在尝试爬取infoID :42370698
Find it!
正在尝试爬取infoID :42863402
Find it!
正在尝试爬取infoID :42773843
Find it!
输出输出
正在尝试爬取infoID :42880070
Find it!
输出输出
正在尝试爬取infoID :42604692
Find it!
输出输出
正在尝试爬取infoID :42706546
Find it!
输出输出
正在尝试爬取infoID :42812191
Find it!
输出输出
正在尝试爬取infoID :42606522
Find it!
输出输出
正在尝试爬取infoID :42880072
Find it!
输出输出
正在尝试爬取infoID :42775840
Find it!
输出输出
正在尝试爬取infoID :42876289
Find it!
正在尝试爬取infoID :41711851
Find it!
输出输出
正在尝试爬取infoID :42634440
Find it!
输

In [135]:
# Implement
# begin = 42751300
# end = 42800000

for index in range(begin, end):
    time.sleep(1 + (random.randint(0,9) / 10))
    print("正在尝试爬取第" + str(index) + "个页面")
    while 1:
        raw_html = get_html(index)
        t_html = etree.HTML(raw_html)
        fail_or_not = t_html.xpath('.//div[@class="fail-title"]')
        if (len(fail_or_not) == 0):
            break
        else :
            print("太快了！准备等待重新爬取第" + str(index) + "个页面")
            time.sleep(10 + random.randint(0,9))


    brand_name = t_html.xpath('.//div[@class="breadnav content"]/a[@target="_blank"]')
    basic_info = t_html.xpath('.//div[@class="source-info-con"]/p')
    price = t_html.xpath('.//div[@class="source-info-con"]/p[@class="sp-margin"]/span')
    
    specid = t_html.xpath('.//input[@id="CarSpecid"]/@value')
    if(len(brand_name) == 0) :
        continue
    print("Find it!")
    
    try:
        # 处理直辖市
        if (len(brand_name) == 7) :
            province = brand_name[1].text
            city = brand_name[2].text.replace("二手车","")
            brand = brand_name[3].text.replace("二手","")
            series = brand_name[4].text.replace("二手","")
            car_name = brand_name[5].text
        else:
            province = brand_name[1].text
            city = province
            brand = brand_name[2].text.replace("二手","")
            series = brand_name[3].text.replace("二手","")
            car_name = brand_name[4].text
            
        
        miles = basic_info[0].text.split('／')[0].replace("万公里","")
        date = basic_info[0].text.split('／')[1]
        price = price[0].text.replace("￥","")
        
        car_config = get_json(specid[0])
        result = {}
        
        for item in car_config["result"]["paramtypeitems"][0]['paramitems']:
            if item['id'] in ID:
                a = item['name']
                b = item['value']
                result[a] = b
        
        # 有些车型信息不全直接丢弃
        if len(result) == len(ID):
            print("输出输出")
            with open("whole_car.csv", 'a', encoding = "utf-8") as f:
                csv_write = csv.writer(f, delimiter=',')
                data_row = [brand, series, car_name, province, city, miles, date, price]
                for data in result.values():
                    data_row.append(data)
                csv_write.writerow(data_row)
    except:
        pass

正在尝试爬取第42751300个页面
正在尝试爬取第42751301个页面
Find it!
输出输出
正在尝试爬取第42751302个页面
正在尝试爬取第42751303个页面
正在尝试爬取第42751304个页面
Find it!
输出输出
正在尝试爬取第42751305个页面
Find it!
输出输出
正在尝试爬取第42751306个页面
Find it!
输出输出
正在尝试爬取第42751307个页面
Find it!
输出输出
正在尝试爬取第42751308个页面
Find it!
正在尝试爬取第42751309个页面
太快了！准备等待重新爬取第42751309个页面
正在尝试爬取第42751310个页面
正在尝试爬取第42751311个页面
正在尝试爬取第42751312个页面
Find it!
输出输出
正在尝试爬取第42751313个页面
Find it!
输出输出
正在尝试爬取第42751314个页面
Find it!
输出输出
正在尝试爬取第42751315个页面
Find it!
输出输出
正在尝试爬取第42751316个页面
太快了！准备等待重新爬取第42751316个页面
太快了！准备等待重新爬取第42751316个页面
正在尝试爬取第42751317个页面
Find it!
输出输出
正在尝试爬取第42751318个页面
太快了！准备等待重新爬取第42751318个页面
正在尝试爬取第42751319个页面
正在尝试爬取第42751320个页面
Find it!
输出输出
正在尝试爬取第42751321个页面
Find it!
输出输出
正在尝试爬取第42751322个页面
Find it!
正在尝试爬取第42751323个页面
Find it!
正在尝试爬取第42751324个页面
太快了！准备等待重新爬取第42751324个页面
太快了！准备等待重新爬取第42751324个页面
正在尝试爬取第42751325个页面
正在尝试爬取第42751326个页面
太快了！准备等待重新爬取第42751326个页面
正在尝试爬取第42751327个页面
太快了！准备等待重新爬取第42751327个页面
正在尝试爬取第42751328个页面
Find it!
输出输出
正在尝试爬取第42751329个页面
太快了！准备等待重新爬取第42751329

Find it!
正在尝试爬取第42751533个页面
正在尝试爬取第42751534个页面
Find it!
输出输出
正在尝试爬取第42751535个页面
Find it!
输出输出
正在尝试爬取第42751536个页面
正在尝试爬取第42751537个页面
太快了！准备等待重新爬取第42751537个页面
正在尝试爬取第42751538个页面
Find it!
输出输出
正在尝试爬取第42751539个页面
Find it!
输出输出
正在尝试爬取第42751540个页面
太快了！准备等待重新爬取第42751540个页面
太快了！准备等待重新爬取第42751540个页面
正在尝试爬取第42751541个页面
正在尝试爬取第42751542个页面
Find it!
输出输出
正在尝试爬取第42751543个页面
太快了！准备等待重新爬取第42751543个页面
正在尝试爬取第42751544个页面
Find it!
输出输出
正在尝试爬取第42751545个页面
太快了！准备等待重新爬取第42751545个页面
正在尝试爬取第42751546个页面
Find it!
输出输出
正在尝试爬取第42751547个页面
Find it!
输出输出
正在尝试爬取第42751548个页面
Find it!
输出输出
正在尝试爬取第42751549个页面
正在尝试爬取第42751550个页面
正在尝试爬取第42751551个页面
太快了！准备等待重新爬取第42751551个页面
太快了！准备等待重新爬取第42751551个页面
正在尝试爬取第42751552个页面
Find it!
输出输出
正在尝试爬取第42751553个页面
Find it!
输出输出
正在尝试爬取第42751554个页面
太快了！准备等待重新爬取第42751554个页面
正在尝试爬取第42751555个页面
正在尝试爬取第42751556个页面
正在尝试爬取第42751557个页面
太快了！准备等待重新爬取第42751557个页面
太快了！准备等待重新爬取第42751557个页面
太快了！准备等待重新爬取第42751557个页面
正在尝试爬取第42751558个页面
Find it!
输出输出
正在尝试爬取第42751559个页面
太快了！准备等待重新爬取第42751559个页面
正在尝试爬取第427

KeyboardInterrupt: 

In [77]:

# 多进程版本
# from multiprocessing import Pool
# from crap import get_html, get_json, run
# import warnings
# warnings.filterwarnings("ignore")

# if __name__ == "__main__":
#     pool = Pool(4)
#     for i in range(42700000, 42800000):
#         pool.apply_async(run, (i, ))

#     pool.close()
#     pool.join()

In [161]:
car_id_html = get_carID("shanghai", 100)
t_car_id_html = etree.HTML(car_id_html)
divs = t_car_id_html.xpath('.//div[@class="tp-cards-tofu fn-clear"]')
infoid = divs[0].xpath('//li/@infoid')

In [72]:
len(brand_name)

0

In [15]:
# 第一次爬取进度 42000000 -- 42113997
# 第二次爬取进度 42800000 -- 42900000
# 第三次爬取进度 42700000 -- 42800000

In [43]:
#测试 v1
raw_html = get_html(42751090)
t_html = etree.HTML(raw_html)

brand_name = t_html.xpath('.//div[@class="breadnav content"]/a[@target="_blank"]')
basic_info = t_html.xpath('.//div[@class="source-info-con"]/p')
price = t_html.xpath('.//div[@class="source-info-con"]/p[@class="sp-margin"]/span')
# basic_parameter = t_html.xpath('.//table[@id="anchorxpath('.//div[@class="source-info-con"]/p[@class="sp-margin"]/span')z10"]')
specid = t_html.xpath('.//input[@id="CarSpecid"]/@value')

In [74]:
# 测试 v2


In [76]:
brand_name

[]

In [45]:
province = brand_name[1].text
city = brand_name[2].text.replace("二手车","")
brand = brand_name[3].text.replace("二手","")
series = brand_name[4].text.replace("二手","")
car_name = brand_name[5].text
miles = basic_info[0].text.split('／')[0].replace("万公里","")
date = basic_info[0].text.split('／')[1]
price = price[0].text.replace("￥","")

In [46]:
print(province)
print(city)
print(brand)
print(series)
print(car_name)
print(miles)
print(date)
print(price)

湖南
长沙
大众
Polo
Polo 2016款 1.4L 自动风尚型
9
2016-09
5.38
